In [27]:
import pandas as pd
import numpy as np

# Load the Excel file
file_path = "D:/IAQ_data/IAQ_Master_file/Cleaned_Duplicates/Without_Outliers/AbruptPeak.xlsx"  # Change this to your actual file path
df = pd.read_excel(file_path, sheet_name='L3')

# Ensure 'date_time' column is in datetime format and sort by time
df['date_time'] = pd.to_datetime(df['date_time'])
df = df.sort_values(by='date_time')

# Compute time difference in minutes
df['time_diff'] = df['date_time'].diff().dt.total_seconds() / 60  # Convert seconds to minutes

# Compute slope: Δpm10 / Δtime
df['slope'] = df['pm10'].diff() / df['time_diff']

# Calculate mean and standard deviation of slopes
mean_slope = df['slope'].mean()
std_slope = df['slope'].std()

# Define threshold for abrupt peaks (3 standard deviations)
threshold = mean_slope + (3 * std_slope)

# Identify abrupt peaks
df['abrupt_peak'] = df['slope'].abs() > threshold

# Separate valid data and abrupt peaks
valid_data = df[~df['abrupt_peak']].drop(columns=['time_diff', 'slope', 'abrupt_peak'])
abrupt_peaks = df[df['abrupt_peak']].drop(columns=['time_diff', 'slope', 'abrupt_peak'])

# Save to Excel with two sheets
output_file = "filtered_data_L3.xlsx"
with pd.ExcelWriter(output_file) as writer:
    valid_data.to_excel(writer, sheet_name="valid_data", index=False)
    abrupt_peaks.to_excel(writer, sheet_name="abrupt_peaks", index=False)

print(f"Processing complete! Valid data saved in '{output_file}' under 'valid_data' sheet.")
print(f"Abrupt peaks saved under 'abrupt_peaks' sheet.")


Processing complete! Valid data saved in 'filtered_data_L3.xlsx' under 'valid_data' sheet.
Abrupt peaks saved under 'abrupt_peaks' sheet.


In [ ]:
#### Hybrid method

In [17]:
import pandas as pd
import numpy as np

# Load the Excel file
file_path = "D:/IAQ_data/IAQ_Master_file/Cleaned_Duplicates/Without_Outliers/AbruptPeak.xlsx"   # Change to actual file path
df = pd.read_excel(file_path, sheet_name='L113')

# Ensure date_time is in datetime format and sort by time
df['date_time'] = pd.to_datetime(df['date_time'])
df = df.sort_values(by='date_time')

# Compute time difference in minutes
df['time_diff'] = df['date_time'].diff().dt.total_seconds() / 60  # Convert seconds to minutes

# Compute slope: Δpm10 / Δtime
df['slope'] = df['pm10'].diff() / df['time_diff']

# Calculate mean and standard deviation of slopes
mean_slope = df['slope'].mean()
std_slope = df['slope'].std()

# Calculate thresholds
steep_threshold = mean_slope + (3 * std_slope)  # 3σ Rule
percentile_threshold = df['slope'].quantile(0.98)  # 99th percentile

# Identify abrupt peaks (if slope exceeds either threshold)
df['abrupt_peak'] = (df['slope'].abs() > steep_threshold) | (df['slope'].abs() > percentile_threshold)

# Short-duration peaks (must last ≤ 5 minutes)
df['short_peak'] = df['time_diff'].shift(-1) <= 6

# Sudden drop condition (next slope is a sharp decrease)
df['sudden_drop'] = df['slope'].shift(-1) < (-steep_threshold)

# Final abrupt peaks: Only if all three conditions are met
df['final_peak'] = df['abrupt_peak'] & df['short_peak'] & df['sudden_drop']

# Separate valid data and abrupt peaks
valid_data = df[~df['final_peak']].drop(columns=['time_diff', 'slope', 'abrupt_peak', 'short_peak', 'sudden_drop', 'final_peak'])
abrupt_peaks = df[df['final_peak']].drop(columns=['time_diff', 'slope', 'abrupt_peak', 'short_peak', 'sudden_drop', 'final_peak'])

# Save to Excel with two sheets
output_file = "filtered_data_hybrid_L113_C.xlsx"
with pd.ExcelWriter(output_file) as writer:
    valid_data.to_excel(writer, sheet_name="valid_data", index=False)
    abrupt_peaks.to_excel(writer, sheet_name="abrupt_peaks", index=False)

print(f"✅ Processing complete!")
print(f"✔️ Valid data saved in '{output_file}' under 'valid_data' sheet.")
print(f"⚠️ Abrupt peaks saved under 'abrupt_peaks' sheet.")


✅ Processing complete!
✔️ Valid data saved in 'filtered_data_hybrid_L113_C.xlsx' under 'valid_data' sheet.
⚠️ Abrupt peaks saved under 'abrupt_peaks' sheet.


In [ ]:
### Hybrid method with rising points

In [ ]:
##### making data per minute, if not.

In [29]:
import pandas as pd
import numpy as np

# Load the data
file_path = "D:/IAQ_data/IAQ_Master_file/Cleaned_Duplicates/Without_Outliers/AbruptPeak.xlsx"  # Update with your file path
df_raw = pd.read_excel(file_path, sheet_name='L113')


# Convert date_time to datetime format and round to the nearest minute
df_raw['date_time'] = pd.to_datetime(df_raw['date_time']).dt.floor('T')

# Aggregate only pm10
df = df_raw.groupby(['date_time', 'location_id'], as_index=False)['pm10'].mean()

# Ensure date_time is in datetime format and sort by time
df['date_time'] = pd.to_datetime(df['date_time'])
df = df.sort_values(by='date_time')

# Compute time difference in minutes
df['time_diff'] = df['date_time'].diff().dt.total_seconds() / 60  # Convert seconds to minutes

# Compute slope: Δpm10 / Δtime
df['slope'] = df['pm10'].diff() / df['time_diff']

# Calculate mean and standard deviation of slopes
mean_slope = df['slope'].mean()
std_slope = df['slope'].std()

# Calculate thresholds
steep_threshold = mean_slope + (3 * std_slope)  # 3σ Rule
percentile_threshold = df['slope'].quantile(0.99)  # 98th percentile

# Identify abrupt peaks (if slope exceeds either threshold)
df['abrupt_peak'] = (df['slope'].abs() > steep_threshold) | (df['slope'].abs() > percentile_threshold)

# Short-duration peaks (must last ≤ 5 minutes)
df['short_peak'] = df['time_diff'].shift(-1) <= 5

# Sudden drop condition (next slope is a sharp decrease)
df['sudden_drop'] = df['slope'].shift(-1) < (-steep_threshold)

# Identify rising points before an abrupt peak
df['rising_point'] = (df['slope'] > steep_threshold) & (df['time_diff'] <= 6) & df['abrupt_peak'].shift(-1)

# Final abrupt peaks: Include both detected peaks and rising points
df['final_peak'] = df['abrupt_peak'] & df['short_peak'] & df['sudden_drop']
df['final_peak'] = df['final_peak'] | df['rising_point']  # Include rising points

# Separate valid data and abrupt peaks
valid_data = df[~df['final_peak']].drop(columns=['time_diff', 'slope', 'abrupt_peak', 'short_peak', 'sudden_drop', 'rising_point', 'final_peak'])
abrupt_peaks = df[df['final_peak']].drop(columns=['time_diff', 'slope', 'abrupt_peak', 'short_peak', 'sudden_drop', 'rising_point', 'final_peak'])

# Save to Excel with two sheets
output_file = "filtered_data_hybridR_L.xlsx"
with pd.ExcelWriter(output_file) as writer:
    valid_data.to_excel(writer, sheet_name="valid_data", index=False)
    abrupt_peaks.to_excel(writer, sheet_name="abrupt_peaks", index=False)

print(f"✅ Processing complete!")
print(f"✔️ Valid data saved in '{output_file}' under 'valid_data' sheet.")
print(f"⚠️ Abrupt peaks (with rising points) saved under 'abrupt_peaks' sheet.")


✅ Processing complete!
✔️ Valid data saved in 'filtered_data_hybridR_L3.xlsx' under 'valid_data' sheet.
⚠️ Abrupt peaks (with rising points) saved under 'abrupt_peaks' sheet.


In [ ]:
### Now taking all datapoints from minute if data is converted from realtime per second data.

In [69]:
import pandas as pd
import numpy as np

# Load the data
file_path = "D:/IAQ_data/IAQ_Master_file/Cleaned_Duplicates/Without_Outliers/AbruptPeak.xlsx"  # Update with your file path
df_raw = pd.read_excel(file_path, sheet_name='L3')

# Convert date_time to datetime format and round to the nearest minute
df_raw['date_time'] = pd.to_datetime(df_raw['date_time']).dt.floor('T')

# Aggregate only pm10
df = df_raw.groupby(['date_time', 'location_id'], as_index=False)['pm10'].mean()

# Ensure date_time is in datetime format and sort by time
df['date_time'] = pd.to_datetime(df['date_time'])
df = df.sort_values(by='date_time')

# Compute time difference in minutes
df['time_diff'] = df['date_time'].diff().dt.total_seconds() / 60  # Convert seconds to minutes

# Compute slope: Δpm10 / Δtime
df['slope'] = df['pm10'].diff() / df['time_diff']

# Calculate mean and standard deviation of slopes
mean_slope = df['slope'].mean()
std_slope = df['slope'].std()

# Calculate thresholds
steep_threshold = mean_slope + (3 * std_slope)  # 3σ Rule
percentile_threshold = df['slope'].quantile(0.99)  # 98th percentile

# Identify abrupt peaks (if slope exceeds either threshold)
df['abrupt_peak'] = (df['slope'].abs() > steep_threshold) | (df['slope'].abs() > percentile_threshold)

# Short-duration peaks (must last ≤ 5 minutes)
df['short_peak'] = df['time_diff'].shift(-1) <= 5

# Sudden drop condition (next slope is a sharp decrease)
df['sudden_drop'] = df['slope'].shift(-1) < (-steep_threshold)

# Identify rising points before an abrupt peak
df['rising_point'] = (df['slope'] > steep_threshold) & (df['time_diff'] <= 6) & df['abrupt_peak'].shift(-1)

# Final abrupt peaks: Include both detected peaks and rising points
df['final_peak'] = df['abrupt_peak'] & df['short_peak'] & df['sudden_drop']
df['final_peak'] = df['final_peak'] | df['rising_point']  # Include rising points

# Extract unique timestamps of abrupt peaks
abrupt_peak_timestamps = df.loc[df['final_peak'], 'date_time'].dt.strftime('%Y-%m-%d %H:%M')

# Filter raw data for all rows within detected abrupt peak minutes
df_raw['minute_str'] = df_raw['date_time'].dt.strftime('%Y-%m-%d %H:%M')
abrupt_peaks_raw = df_raw[df_raw['minute_str'].isin(abrupt_peak_timestamps)]

# Drop auxiliary column
df_raw.drop(columns=['minute_str'], inplace=True)

# Separate valid data
df_valid = df[~df['final_peak']].drop(columns=['time_diff', 'slope', 'abrupt_peak', 'short_peak', 'sudden_drop', 'rising_point', 'final_peak'])

# Save to Excel with two sheets
output_file = "filtered_data_hybridR_L3_persecond.xlsx"
with pd.ExcelWriter(output_file) as writer:
    df_valid.to_excel(writer, sheet_name="valid_data", index=False)
    abrupt_peaks_raw.to_excel(writer, sheet_name="abrupt_peaks", index=False)

print(f"✅ Processing complete!")
print(f"✔️ Valid data saved in '{output_file}' under 'valid_data' sheet.")
print(f"⚠️ Abrupt peaks (all raw data from those minutes) saved under 'abrupt_peaks' sheet.")


✅ Processing complete!
✔️ Valid data saved in 'filtered_data_hybridR_L3_persecond.xlsx' under 'valid_data' sheet.
⚠️ Abrupt peaks (all raw data from those minutes) saved under 'abrupt_peaks' sheet.


In [ ]:
#### Hybrid mwthod with background slope of 3 hours

In [71]:
import pandas as pd
import numpy as np

# Load the data
file_path = "D:/IAQ_data/IAQ_Master_file/Cleaned_Duplicates/Without_Outliers/AbruptPeak.xlsx"  # Update with your file path
df_raw = pd.read_excel(file_path, sheet_name='L3')

# Convert date_time to datetime format and round to the nearest minute
df_raw['date_time'] = pd.to_datetime(df_raw['date_time']).dt.floor('T')

# Aggregate only pm10
df = df_raw.groupby(['date_time', 'location_id'], as_index=False)['pm10'].mean()

# Ensure date_time is in datetime format and sort by time
df['date_time'] = pd.to_datetime(df['date_time'])
df = df.sort_values(by='date_time')

# Compute time difference in minutes
df['time_diff'] = df['date_time'].diff().dt.total_seconds() / 60  # Convert seconds to minutes

# Compute slope: Δpm10 / Δtime
df['slope'] = df['pm10'].diff() / df['time_diff']

# Function to compute background slope over the last 3 hours
def compute_background_slope(df):
    background_slopes = []
    for i in range(len(df)):
        start_time = df.loc[i, 'date_time'] - pd.Timedelta(hours=3)
        subset = df[(df['date_time'] >= start_time) & (df['date_time'] <= df.loc[i, 'date_time'])]
        if len(subset) < 2:  # If not enough data, look ahead
            end_time = df.loc[i, 'date_time'] + pd.Timedelta(hours=(3 - (df.loc[i, 'date_time'] - df['date_time'].min()).total_seconds() / 3600))
            subset = df[(df['date_time'] >= df.loc[i, 'date_time']) & (df['date_time'] <= end_time)]
        background_slopes.append(subset['slope'].mean())
    return np.array(background_slopes)

# Compute background slope
df['background_slope'] = compute_background_slope(df)

# Calculate thresholds dynamically per data point
steep_threshold = df['background_slope'] + (3 * df['slope'].std())
percentile_threshold = df['slope'].rolling(window=180, min_periods=1).quantile(0.99)  # 3 hours ~ 180 minutes

# Identify abrupt peaks (if slope exceeds either threshold)
df['abrupt_peak'] = (df['slope'].abs() > steep_threshold) | (df['slope'].abs() > percentile_threshold)

# Short-duration peaks (must last ≤ 5 minutes)
df['short_peak'] = df['time_diff'].shift(-1) <= 5

# Sudden drop condition (next slope is a sharp decrease)
df['sudden_drop'] = df['slope'].shift(-1) < (-steep_threshold)

# Identify rising points before an abrupt peak
df['rising_point'] = (df['slope'] > steep_threshold) & (df['time_diff'] <= 6) & df['abrupt_peak'].shift(-1)

# Final abrupt peaks: Include both detected peaks and rising points
df['final_peak'] = df['abrupt_peak'] & df['short_peak'] & df['sudden_drop']
df['final_peak'] = df['final_peak'] | df['rising_point']  # Include rising points

# Extract unique timestamps of abrupt peaks
abrupt_peak_timestamps = df.loc[df['final_peak'], 'date_time'].dt.strftime('%Y-%m-%d %H:%M')

# Filter raw data for all rows within detected abrupt peak minutes
df_raw['minute_str'] = df_raw['date_time'].dt.strftime('%Y-%m-%d %H:%M')
abrupt_peaks_raw = df_raw[df_raw['minute_str'].isin(abrupt_peak_timestamps)]

# Drop auxiliary column
df_raw.drop(columns=['minute_str'], inplace=True)

# Separate valid data
df_valid = df[~df['final_peak']].drop(columns=['time_diff', 'slope', 'abrupt_peak', 'short_peak', 'sudden_drop', 'rising_point', 'final_peak', 'background_slope'])

# Save to Excel with two sheets
output_file = "filtered_data_hybridR_L3_background.xlsx"
with pd.ExcelWriter(output_file) as writer:
    df_valid.to_excel(writer, sheet_name="valid_data", index=False)
    abrupt_peaks_raw.to_excel(writer, sheet_name="abrupt_peaks", index=False)

print(f"✅ Processing complete!")
print(f"✔️ Valid data saved in '{output_file}' under 'valid_data' sheet.")
print(f"⚠️ Abrupt peaks (all raw data from those minutes) saved under 'abrupt_peaks' sheet.")


✅ Processing complete!
✔️ Valid data saved in 'filtered_data_hybridR_L3_background.xlsx' under 'valid_data' sheet.
⚠️ Abrupt peaks (all raw data from those minutes) saved under 'abrupt_peaks' sheet.
